In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [9]:
dataset1 = pd.read_csv('C:/Users/taochan/Desktop/X-code/X-data/ExtractFeature/dataset1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('C:/Users/taochan/Desktop/X-code/X-data/ExtractFeature/dataset2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('C:/Users/taochan/Desktop/X-code/X-data/ExtractFeature/dataset3.csv')

dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)

dataset12 = pd.concat([dataset1,dataset2],axis=0)

dataset1_y = dataset1.label
dataset1_x = dataset1.drop(['User_id','label','day_gap_before','day_gap_after'],axis=1)  # 'day_gap_before','day_gap_after' cause overfitting, 0.77
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['User_id','label','day_gap_before','day_gap_after'],axis=1)
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['User_id','label','day_gap_before','day_gap_after'],axis=1)
dataset3_preds = dataset3[['User_id','Coupon_id','Date_received']]
dataset3_x = dataset3.drop(['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after'],axis=1)

print(dataset1_x.shape,dataset2_x.shape,dataset3_x.shape)

dataset1 = xgb.DMatrix(dataset1_x,label=dataset1_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset12 = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataset3 = xgb.DMatrix(dataset3_x)

params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

#train on dataset1, evaluate on dataset2
#watchlist = [(dataset1,'train'),(dataset2,'val')]
#model = xgb.train(params,dataset1,num_boost_round=3000,evals=watchlist,early_stopping_rounds=300)

watchlist = [(dataset12,'train')]
model = xgb.train(params,dataset12,num_boost_round=3500,evals=watchlist)

#predict test set
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds[['label']])
dataset3_preds.sort_values(by=['Coupon_id','label'],inplace=True)
dataset3_preds.to_csv("C:/Users/taochan/Desktop/X-code/X-data/result/xgb_preds.csv",index=None,header=None)
print(dataset3_preds.describe())
    
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('C:/Users/taochan/Desktop/X-code/X-data/result/xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)

(129123, 51) (251978, 51) (112803, 51)
[0]	train-auc:0.807918
[1]	train-auc:0.822091
[2]	train-auc:0.826246
[3]	train-auc:0.82923
[4]	train-auc:0.834726
[5]	train-auc:0.835788
[6]	train-auc:0.83631
[7]	train-auc:0.837404
[8]	train-auc:0.837826
[9]	train-auc:0.838439
[10]	train-auc:0.839478
[11]	train-auc:0.839788
[12]	train-auc:0.839908
[13]	train-auc:0.840791
[14]	train-auc:0.841188
[15]	train-auc:0.841136
[16]	train-auc:0.841804
[17]	train-auc:0.841689
[18]	train-auc:0.84209
[19]	train-auc:0.842449
[20]	train-auc:0.842179
[21]	train-auc:0.842296
[22]	train-auc:0.842325
[23]	train-auc:0.842267
[24]	train-auc:0.842085
[25]	train-auc:0.842144
[26]	train-auc:0.842053
[27]	train-auc:0.842117
[28]	train-auc:0.842415
[29]	train-auc:0.84295
[30]	train-auc:0.842952
[31]	train-auc:0.843047
[32]	train-auc:0.842972
[33]	train-auc:0.842868
[34]	train-auc:0.843141
[35]	train-auc:0.843118
[36]	train-auc:0.843146
[37]	train-auc:0.843422
[38]	train-auc:0.843507
[39]	train-auc:0.843622
[40]	train-auc:

[333]	train-auc:0.855032
[334]	train-auc:0.855062
[335]	train-auc:0.855096
[336]	train-auc:0.855155
[337]	train-auc:0.855173
[338]	train-auc:0.855209
[339]	train-auc:0.855234
[340]	train-auc:0.855278
[341]	train-auc:0.85531
[342]	train-auc:0.855344
[343]	train-auc:0.855383
[344]	train-auc:0.855419
[345]	train-auc:0.855451
[346]	train-auc:0.855486
[347]	train-auc:0.855508
[348]	train-auc:0.855521
[349]	train-auc:0.855568
[350]	train-auc:0.855598
[351]	train-auc:0.855629
[352]	train-auc:0.855661
[353]	train-auc:0.855693
[354]	train-auc:0.85572
[355]	train-auc:0.855737
[356]	train-auc:0.855758
[357]	train-auc:0.855779
[358]	train-auc:0.855805
[359]	train-auc:0.855832
[360]	train-auc:0.855843
[361]	train-auc:0.855874
[362]	train-auc:0.855896
[363]	train-auc:0.855929
[364]	train-auc:0.855997
[365]	train-auc:0.856016
[366]	train-auc:0.856043
[367]	train-auc:0.856064
[368]	train-auc:0.856082
[369]	train-auc:0.856098
[370]	train-auc:0.856136
[371]	train-auc:0.85616
[372]	train-auc:0.856179
[37

[663]	train-auc:0.86265
[664]	train-auc:0.862666
[665]	train-auc:0.862682
[666]	train-auc:0.862691
[667]	train-auc:0.8627
[668]	train-auc:0.862717
[669]	train-auc:0.862734
[670]	train-auc:0.86275
[671]	train-auc:0.862774
[672]	train-auc:0.862782
[673]	train-auc:0.862795
[674]	train-auc:0.86281
[675]	train-auc:0.86286
[676]	train-auc:0.862877
[677]	train-auc:0.862896
[678]	train-auc:0.86294
[679]	train-auc:0.862951
[680]	train-auc:0.862975
[681]	train-auc:0.863003
[682]	train-auc:0.86302
[683]	train-auc:0.863042
[684]	train-auc:0.863055
[685]	train-auc:0.863069
[686]	train-auc:0.863093
[687]	train-auc:0.86313
[688]	train-auc:0.86316
[689]	train-auc:0.863192
[690]	train-auc:0.863204
[691]	train-auc:0.863224
[692]	train-auc:0.863256
[693]	train-auc:0.86327
[694]	train-auc:0.863289
[695]	train-auc:0.863302
[696]	train-auc:0.863308
[697]	train-auc:0.863329
[698]	train-auc:0.863338
[699]	train-auc:0.863351
[700]	train-auc:0.86337
[701]	train-auc:0.86339
[702]	train-auc:0.86341
[703]	train-au

[993]	train-auc:0.868171
[994]	train-auc:0.868184
[995]	train-auc:0.868201
[996]	train-auc:0.868226
[997]	train-auc:0.868245
[998]	train-auc:0.86826
[999]	train-auc:0.868273
[1000]	train-auc:0.86828
[1001]	train-auc:0.868295
[1002]	train-auc:0.86831
[1003]	train-auc:0.868326
[1004]	train-auc:0.868339
[1005]	train-auc:0.868355
[1006]	train-auc:0.868375
[1007]	train-auc:0.86839
[1008]	train-auc:0.868397
[1009]	train-auc:0.868408
[1010]	train-auc:0.868432
[1011]	train-auc:0.868447
[1012]	train-auc:0.868458
[1013]	train-auc:0.868471
[1014]	train-auc:0.868488
[1015]	train-auc:0.868497
[1016]	train-auc:0.868516
[1017]	train-auc:0.868525
[1018]	train-auc:0.868533
[1019]	train-auc:0.868549
[1020]	train-auc:0.868554
[1021]	train-auc:0.868572
[1022]	train-auc:0.868587
[1023]	train-auc:0.868614
[1024]	train-auc:0.868623
[1025]	train-auc:0.868642
[1026]	train-auc:0.868652
[1027]	train-auc:0.868663
[1028]	train-auc:0.868679
[1029]	train-auc:0.868692
[1030]	train-auc:0.868698
[1031]	train-auc:0.8687

[1310]	train-auc:0.872287
[1311]	train-auc:0.872292
[1312]	train-auc:0.8723
[1313]	train-auc:0.872307
[1314]	train-auc:0.872316
[1315]	train-auc:0.872334
[1316]	train-auc:0.872344
[1317]	train-auc:0.872355
[1318]	train-auc:0.872372
[1319]	train-auc:0.872383
[1320]	train-auc:0.872393
[1321]	train-auc:0.872407
[1322]	train-auc:0.872421
[1323]	train-auc:0.872431
[1324]	train-auc:0.872436
[1325]	train-auc:0.872445
[1326]	train-auc:0.872452
[1327]	train-auc:0.872468
[1328]	train-auc:0.87248
[1329]	train-auc:0.872489
[1330]	train-auc:0.872498
[1331]	train-auc:0.872509
[1332]	train-auc:0.872515
[1333]	train-auc:0.872523
[1334]	train-auc:0.872533
[1335]	train-auc:0.872537
[1336]	train-auc:0.872546
[1337]	train-auc:0.872553
[1338]	train-auc:0.872572
[1339]	train-auc:0.872589
[1340]	train-auc:0.872595
[1341]	train-auc:0.872599
[1342]	train-auc:0.872615
[1343]	train-auc:0.87263
[1344]	train-auc:0.872639
[1345]	train-auc:0.872652
[1346]	train-auc:0.872666
[1347]	train-auc:0.872677
[1348]	train-auc

...

[1944]	train-auc:0.878003
[1945]	train-auc:0.87801
[1946]	train-auc:0.878017
[1947]	train-auc:0.878028
[1948]	train-auc:0.878036
[1949]	train-auc:0.878041
[1950]	train-auc:0.878046
[1951]	train-auc:0.878055
[1952]	train-auc:0.878059
[1953]	train-auc:0.878066
[1954]	train-auc:0.878072
[1955]	train-auc:0.878078
[1956]	train-auc:0.878085
[1957]	train-auc:0.878091
[1958]	train-auc:0.878103
[1959]	train-auc:0.87811
[1960]	train-auc:0.878114
[1961]	train-auc:0.878121
[1962]	train-auc:0.87813
[1963]	train-auc:0.878138
[1964]	train-auc:0.878147
[1965]	train-auc:0.878155
[1966]	train-auc:0.878164
[1967]	train-auc:0.878167
[1968]	train-auc:0.878175
[1969]	train-auc:0.878178
[1970]	train-auc:0.878185
[1971]	train-auc:0.878194
[1972]	train-auc:0.878199
[1973]	train-auc:0.878205
[1974]	train-auc:0.87821
[1975]	train-auc:0.878219
[1976]	train-auc:0.87823
[1977]	train-auc:0.878238
[1978]	train-auc:0.878247
[1979]	train-auc:0.878258
[1980]	train-auc:0.878267
[1981]	train-auc:0.878277
[1982]	train-auc:

[2261]	train-auc:0.880226
[2262]	train-auc:0.880234
[2263]	train-auc:0.88024
[2264]	train-auc:0.880246
[2265]	train-auc:0.880254
[2266]	train-auc:0.880259
[2267]	train-auc:0.880262
[2268]	train-auc:0.880268
[2269]	train-auc:0.880276
[2270]	train-auc:0.880282
[2271]	train-auc:0.880291
[2272]	train-auc:0.880299
[2273]	train-auc:0.880307
[2274]	train-auc:0.880314
[2275]	train-auc:0.880321
[2276]	train-auc:0.880328
[2277]	train-auc:0.880334
[2278]	train-auc:0.880339
[2279]	train-auc:0.880347
[2280]	train-auc:0.880354
[2281]	train-auc:0.88036
[2282]	train-auc:0.880369
[2283]	train-auc:0.880374
[2284]	train-auc:0.880378
[2285]	train-auc:0.880382
[2286]	train-auc:0.880392
[2287]	train-auc:0.880399
[2288]	train-auc:0.880406
[2289]	train-auc:0.880414
[2290]	train-auc:0.88042
[2291]	train-auc:0.880429
[2292]	train-auc:0.880435
[2293]	train-auc:0.880441
[2294]	train-auc:0.880447
[2295]	train-auc:0.880458
[2296]	train-auc:0.880465
[2297]	train-auc:0.88047
[2298]	train-auc:0.880475
[2299]	train-auc

[2578]	train-auc:0.882239
[2579]	train-auc:0.882243
[2580]	train-auc:0.882247
[2581]	train-auc:0.882254
[2582]	train-auc:0.882259
[2583]	train-auc:0.882267
[2584]	train-auc:0.882274
[2585]	train-auc:0.882278
[2586]	train-auc:0.882284
[2587]	train-auc:0.882291
[2588]	train-auc:0.8823
[2589]	train-auc:0.882306
[2590]	train-auc:0.882309
[2591]	train-auc:0.882316
[2592]	train-auc:0.88232
[2593]	train-auc:0.882329
[2594]	train-auc:0.882332
[2595]	train-auc:0.882336
[2596]	train-auc:0.882345
[2597]	train-auc:0.882349
[2598]	train-auc:0.882355
[2599]	train-auc:0.882362
[2600]	train-auc:0.882369
[2601]	train-auc:0.882374
[2602]	train-auc:0.882379
[2603]	train-auc:0.882388
[2604]	train-auc:0.882396
[2605]	train-auc:0.882402
[2606]	train-auc:0.882409
[2607]	train-auc:0.882415
[2608]	train-auc:0.882421
[2609]	train-auc:0.882428
[2610]	train-auc:0.882431
[2611]	train-auc:0.882437
[2612]	train-auc:0.882439
[2613]	train-auc:0.882446
[2614]	train-auc:0.882451
[2615]	train-auc:0.882454
[2616]	train-au

[2895]	train-auc:0.883992
[2896]	train-auc:0.884
[2897]	train-auc:0.884006
[2898]	train-auc:0.884009
[2899]	train-auc:0.884015
[2900]	train-auc:0.884021
[2901]	train-auc:0.884024
[2902]	train-auc:0.88403
[2903]	train-auc:0.884035
[2904]	train-auc:0.884039
[2905]	train-auc:0.884043
[2906]	train-auc:0.884049
[2907]	train-auc:0.884053
[2908]	train-auc:0.884055
[2909]	train-auc:0.884062
[2910]	train-auc:0.88407
[2911]	train-auc:0.884077
[2912]	train-auc:0.884084
[2913]	train-auc:0.884088
[2914]	train-auc:0.884097
[2915]	train-auc:0.884103
[2916]	train-auc:0.884109
[2917]	train-auc:0.884115
[2918]	train-auc:0.88412
[2919]	train-auc:0.884126
[2920]	train-auc:0.88413
[2921]	train-auc:0.884135
[2922]	train-auc:0.884141
[2923]	train-auc:0.884146
[2924]	train-auc:0.884151
[2925]	train-auc:0.884154
[2926]	train-auc:0.884161
[2927]	train-auc:0.884165
[2928]	train-auc:0.884171
[2929]	train-auc:0.884174
[2930]	train-auc:0.884181
[2931]	train-auc:0.884186
[2932]	train-auc:0.884191
[2933]	train-auc:0.

[3212]	train-auc:0.88563
[3213]	train-auc:0.885638
[3214]	train-auc:0.885644
[3215]	train-auc:0.885647
[3216]	train-auc:0.885651
[3217]	train-auc:0.885655
[3218]	train-auc:0.885659
[3219]	train-auc:0.885664
[3220]	train-auc:0.885669
[3221]	train-auc:0.885675
[3222]	train-auc:0.885679
[3223]	train-auc:0.885685
[3224]	train-auc:0.885691
[3225]	train-auc:0.885698
[3226]	train-auc:0.885702
[3227]	train-auc:0.885704
[3228]	train-auc:0.885711
[3229]	train-auc:0.885713
[3230]	train-auc:0.885717
[3231]	train-auc:0.885723
[3232]	train-auc:0.885728
[3233]	train-auc:0.885735
[3234]	train-auc:0.88574
[3235]	train-auc:0.885745
[3236]	train-auc:0.885749
[3237]	train-auc:0.885752
[3238]	train-auc:0.885756
[3239]	train-auc:0.885759
[3240]	train-auc:0.885764
[3241]	train-auc:0.885767
[3242]	train-auc:0.885771
[3243]	train-auc:0.885777
[3244]	train-auc:0.885779
[3245]	train-auc:0.885783
[3246]	train-auc:0.885788
[3247]	train-auc:0.885792
[3248]	train-auc:0.885795
[3249]	train-auc:0.885799
[3250]	train-a